In [1]:
import sys
sys.path.append("..")  # <-  This should point to the root directory of the project relative to this file

from scripts import database

In [2]:
example_query =  """
                SELECT * FROM flights
                    LIMIT 10;
                """

example = database.query(example_query)

In [3]:
example

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2019-01-01,AA,AA,AA,938,AA,N811NN,938,11697,FLL,...,1182.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
1,2019-01-01,AA,AA,AA,943,AA,N918NN,943,13303,MIA,...,1107.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
2,2019-01-01,AA,AA,AA,943,AA,N918NN,943,15024,STT,...,1107.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
3,2019-01-01,AA,AA,AA,944,AA,N9016,944,15016,STL,...,1068.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
4,2019-01-01,AA,AA,AA,952,AA,N941NN,952,12478,JFK,...,2153.0,2.0,0.0,31.0,0.0,0.0,None,NaN,NaN,None
5,2019-01-01,AA,AA,AA,952,AA,N965NN,952,14107,PHX,...,1009.0,NaN,NaN,NaN,NaN,NaN,1913.0,38.0,38.0,None
6,2019-01-01,AA,AA,AA,955,AA,N944NN,955,11057,CLT,...,728.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
7,2019-01-01,AA,AA,AA,958,AA,N147AA,958,10423,AUS,...,1242.0,0.0,0.0,19.0,0.0,0.0,None,NaN,NaN,None
8,2019-01-01,AA,AA,AA,958,AA,N147AA,958,12892,LAX,...,1242.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
9,2019-01-01,AA,AA,AA,969,AA,N982NN,969,12478,JFK,...,2248.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None


In [4]:
import pandas as pd

In [5]:
FLIGHTS_QUERY =  """
                SELECT * FROM flights
                    WHERE SUBSTRING(fl_date,6,2) = '01'
                    ;
                """

flights = database.query(FLIGHTS_QUERY)
flight_numbers = pd.read_csv('../data/preprocessing/flight_numbers.csv')
unique_carriers = pd.read_csv('../data/preprocessing/unique_carriers.csv')





In [ ]:
flight_numbers

Error: Session cannot generate requests

In [ ]:
unique_carriers

Error: Session cannot generate requests